In [ ]:
import os

import networkx as nx
from IPython.display import Image
from networkx.drawing.nx_agraph import to_agraph

# Ensure the assets directory exists
prefix = "aufgabe_05"
basepath = os.path.join('assets', 'aufgabe_05')
os.makedirs(basepath, exist_ok=True)

# Aufgabe 05

In [ ]:
graph = nx.Graph()
graph.add_nodes_from([
    ('s', {'heuristic': 45}),
    ('a', {'heuristic': 40}),
    ('b', {'heuristic': 35}),
    ('c', {'heuristic': 38}),
    ('d', {'heuristic': 48}),
    ('e', {'heuristic': 55}),
    ('f', {'heuristic': 50}),
    ('g', {'heuristic': 21}),
    ('h', {'heuristic': 20}),
    ('i', {'heuristic': 26}),
    ('z', {'heuristic': 0}),
])

graph.add_edges_from([
    ('s', 'a',  {'weight': 11}),
    ('s', 'b',  {'weight': 10}),
    ('s', 'c',  {'weight': 7}),
    ('s', 'd',  {'weight': 7}),
    ('s', 'e',  {'weight': 10}),
    ('s', 'f',  {'weight': 8}),
    ('a', 'b',  {'weight': 7}),
    ('a', 'f',  {'weight': 10}),
    ('a', 'g',  {'weight': 22}),
    ('b', 'c',  {'weight': 8}),
    ('b', 'g',  {'weight': 19}),
    ('b', 'i',  {'weight': 12}),
    ('c', 'd',  {'weight': 10}),
    ('c', 'i',  {'weight': 12}),
    ('d', 'e',  {'weight': 9}),
    ('e', 'f',  {'weight': 9}),
    ('g', 'h',  {'weight': 8}),
    ('g', 'z',  {'weight': 24}),
    ('h', 'i',  {'weight': 9}),
    ('h', 'z',  {'weight': 22}),
    ('i', 'z',  {'weight': 32}),
])

In [ ]:
def draw_step(a, n, e, filename):
    # Visualize Step
    a.get_node(n).attr['color'] = 'red'
    if e:
        a.get_edge(e[0], e[1]).attr['color'] = 'red'
    a.draw(filename, format='png', prog='dot')
    display(Image(filename))

In [ ]:
def color_gradient(start_color, end_color, index, steps):
    r = int(start_color[0] + (end_color[0] - start_color[0]) * index / (steps - 1))
    g = int(start_color[1] + (end_color[1] - start_color[1]) * index / (steps - 1))
    b = int(start_color[2] + (end_color[2] - start_color[2]) * index / (steps - 1))
    return f'#{hex(r).replace("0x", ""):02}{hex(g).replace("0x", ""):02}{hex(b).replace("0x", ""):02}'

# Djikstra

In [ ]:
def dijkstra(graph: nx.Graph, start: str, end: str):
    F = set()
    dist = {node: float('inf') for node in graph.nodes()}
    W = {node: [] for node in graph.nodes()}
    dist[start] = 0
    W[start] = [start]
    
    while end not in F:
        # Find the node with the smallest distance
        v_star = min((node for node in dist if node not in F), key=lambda node: dist[node])

        # Add this node to the set of finished nodes
        F.add(v_star)

        # Debug output for each iteration
        print(f"F: {F} - dist: {dist} - W: {W}")

        for neighbor in graph.neighbors(v_star):
            if neighbor not in F:
                new_dist = dist[v_star] + graph[v_star][neighbor].get('weight', 1)
                if new_dist < dist[neighbor]:
                    dist[neighbor] = new_dist
                    W[neighbor] = W[v_star] + [neighbor]

    return W[end]

# A*

In [ ]:
def astar(graph: nx.Graph, start: str, end: str):
    F = set()
    dist = {node: float('inf') for node in graph.nodes()}
    W = {node: [] for node in graph.nodes()}
    dist[start] = 0
    W[start] = [start]
    
    while end not in F:
        # Find the node with the smallest distance + heuristic
        v_star = min((node for node in dist if node not in F), key=lambda node: dist[node] + graph.nodes[node].get('heuristic', 0))
    
        # Add this node to the set of finished nodes
        F.add(v_star)
    
        # Debug output for each iteration
        print(f"F: {F} - dist: {dist} - W: {W}")
    
        for neighbor in graph.neighbors(v_star):
            if neighbor not in F:
                new_dist = dist[v_star] + graph[v_star][neighbor].get('weight', 1)
                if new_dist < dist[neighbor]:
                    dist[neighbor] = new_dist
                    W[neighbor] = W[v_star] + [neighbor]
    
    return W[end]

# Executions

In [ ]:
route = dijkstra(graph, 's', 'z')

In [ ]:
dg = to_agraph(graph)
for idx, node in enumerate(route):
    color = color_gradient((255, 0, 0), (142, 0, 255), idx, len(route))
    dg.get_node(node).attr['color'] = color

for idx in range(len(route) - 1):
    edge = dg.get_edge(route[idx], route[idx + 1])
    color = color_gradient((255, 0, 0), (142, 0, 255), idx, len(route) - 1)
    edge.attr['color'] = color
    edge.attr['penwidth'] = 2.0
    edge.attr['label'] = graph.get_edge_data(route[idx], route[idx + 1])['weight']

filename = os.path.join(basepath, 'djikstra.png')
dg.draw(filename, format='png', prog='dot')
display(Image(filename))

In [ ]:
route = astar(graph, 's', 'z')

In [ ]:
ast = to_agraph(graph)
for idx, node in enumerate(route):
    color = color_gradient((255, 0, 0), (142, 0, 255), idx, len(route))
    ast.get_node(node).attr['color'] = color

for idx in range(len(route) - 1):
    edge = ast.get_edge(route[idx], route[idx + 1])
    color = color_gradient((255, 0, 0), (142, 0, 255), idx, len(route) - 1)
    edge.attr['color'] = color
    edge.attr['penwidth'] = 2.0
    edge.attr['label'] = graph.get_edge_data(route[idx], route[idx + 1])['weight']

filename = os.path.join(basepath, 'astar.png')
ast.draw(filename, format='png', prog='dot')
display(Image(filename))